# 1st Scripting Tutorial in Autoscript
- Check vacuum
- Open valves
- connect
- Query state of microscope
    - stage
    - Beam state
    - Beam(paused position)
    - Mode
    - current
- Get HAADF
- Get EDX
- close vacuum valves

In [ ]:
from autoscript_tem_microscope_client import TemMicroscopeClient

# from autoscript_tem_microscope_client.structures import 

import math
import matplotlib.pyplot as plt

In [ ]:
ip = "10.46.217.241"
port = 9095
microscope = TemMicroscopeClient()
microscope.connect(ip, port = port)# 7521 on velox  computer

In [ ]:
microscope.vacuum.state

In [57]:
microscope.vacuum.column_valves.state

'Closed'

In [ ]:
microscope.vacuum.column_valves.open()

In [ ]:
microscope.specimen.stage.position

In [ ]:
microscope.optics.is_beam_blanked

In [ ]:
microscope.optics.paused_scan_beam_position

In [ ]:
microscope.detectors.screen.measure_current() 

In [ ]:
from autoscript_tem_microscope_client.enumerations import  DetectorType
image = microscope.acquisition.acquire_stem_image(scanning_detector=DetectorType.HAADF, size=256, dwell_time=1000e-9)

In [ ]:
import matplotlib.pyplot as plt

plt.imshow(image.data)
plt.scatter

In [ ]:
# metadata of the haadf
print(image.metadata.optics.defocus)
print(image.metadata.optics.c2_lens_intensity)
print(image.metadata.optics.beam_tilt)
print(image.metadata.stage_settings.stage_position)



In [ ]:
# can export - 
# print(image.metadata.metadata_as_xml)

In [ ]:
# check beam position
beam_pos = microscope.optics.paused_scan_beam_position
print(beam_pos)
plt.imshow(image.data)
plt.scatter(beam_pos.x*image.data.shape[0], beam_pos.y*image.data.shape[0], c= "red")

In [ ]:
# put beam at center of the screen 
microscope.optics.paused_scan_beam_position = [0.5, 0.5]

In [ ]:
# check beam position
beam_pos = microscope.optics.paused_scan_beam_position
print(beam_pos)
plt.imshow(image.data)
plt.scatter(beam_pos.x*image.data.shape[0], beam_pos.y*image.data.shape[0], c= "red")

In [ ]:
## Edx
# Acquire single EDS spectrum and plot it

from autoscript_tem_microscope_client.structures import  EdsAcquisitionSettings
from autoscript_tem_microscope_client.enumerations import DetectorType, ExposureTimeType


# Get the first EDS detector
eds_detector_name = microscope.detectors.eds_detectors[0]
eds_detector = microscope.detectors.get_eds_detector(eds_detector_name)

# Configure the acquisition
settings = EdsAcquisitionSettings()
settings.eds_detector = eds_detector_name
settings.dispersion = eds_detector.dispersions[-1]
settings.shaping_time = eds_detector.shaping_times[-1]
settings.exposure_time = 2
settings.exposure_time_type = ExposureTimeType.LIVE_TIME

# Run the spectrum acquisition
spectrum = microscope.analysis.eds.acquire_spectrum(settings)





In [ ]:
import xmltodict
import json
## now with the spectrum lets do the scalrizer stuff we did with dkl
def get_dispersion_and_offset(spectrum):
    """
    Extract dispersion and offset from EDS spectrum metadata (xml).
    Returns (dispersion_keV_per_ch, offset_keV).
    """
    xml_string = spectrum.metadata.metadata_as_xml
    metadata = xmltodict.parse(xml_string)
    metadata = json.loads(json.dumps(metadata))

    detectors = metadata["Metadata"]["Detectors"]["AnalyticalDetector"]

    # If only one detector, wrap it into a list
    if isinstance(detectors, dict):
        detectors = [detectors]

    # Take the first detector (or filter by name if needed)
    det = detectors[0]
    dispersion = float(det.get("Dispersion", 0))
    offset = float(det.get("OffsetEnergy", 0))

    return dispersion, offset

import numpy as np
# Average spectrum data from 4 detectors
n_channels_per_detector = len(spectrum.data) // 4
summed_spectrum = np.zeros(n_channels_per_detector)

for i in range(4):
    start_idx = i * n_channels_per_detector
    end_idx = (i + 1) * n_channels_per_detector
    summed_spectrum += spectrum.data[start_idx:end_idx]

# Use summed spectrum for analysis
spectrum_data = summed_spectrum

# Plot spectrum using matplotlib instead of vision_toolkit
dispersion, offset = get_dispersion_and_offset(spectrum)
energy_axis = (np.arange(len(spectrum_data)) * dispersion + offset)/1000 # 1000 for Kev

plt.figure(figsize=(12, 6))
plt.plot(energy_axis, spectrum_data)
plt.xlabel('Energy (keV)')
plt.ylabel('Counts')
plt.title('EDS Spectrum (Summed from 4 Detectors)')
plt.xlim(0, 20)  # Focus on physically relevant energy range